In [1]:
import numpy as np
from PIL import ImageGrab
import cv2
import time
from numpy import ones,vstack
from numpy.linalg import lstsq
from directkeys import PressKey, W, A, S, D
from statistics import mean
import os
from scipy import optimize
from matplotlib import pyplot as plt, cm, colors
from directkeys import PressKey,ReleaseKey, W, A, S, D
import random

In [2]:
def region_of_interest(image):#enclosed region of our field of view and recall that the enclosed region was triangular in shape
    # height = image.shape[0] #limit the extent of our field of view
    polygons = np.array([[
        [10,500],[10,300], [300,200], [500,200], [800,300], [800,500]]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [3]:
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:#lines=[[[2,3,2,3]]]
            # if line is not None and len(line) == 4:
            x1, y1, x2, y2 = line
            try:
                x1, y1, x2, y2 = x1.astype(int), y1.astype(int), x2.astype(int), y2.astype(int)
                
                #(image, start_point, end_point, color, thickness
                cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 30)
            except ValueError as e:
                print(f"Error converting line coordinates to integers: {e}")
    else:
        pass
       
    return line_image

In [4]:
def canny(image):
    gray=cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur=cv2.GaussianBlur(gray,(5,5),0) #kernel, deviation
    canny=cv2.Canny(blur, 200, 200)
    return canny

In [5]:
def make_coordinates(image,line_parameters):
    slope,intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1,y1,x2,y2])

In [6]:
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    if lines is None:
        return np.array([])

    for line in lines:
        x1, y1, x2, y2 = line
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))

    #operate vertically along the rows to get the average slope and the average y-intercept respectively
    left_line = make_coordinates(image, np.average(left_fit, axis=0)) if left_fit else None
    #average slope, y-intercept of a single line on right side
    right_line = make_coordinates(image, np.average(right_fit, axis=0)) if right_fit else None

    lines_to_return = []
    if left_line is not None:
        lines_to_return.append(left_line)
    if right_line is not None:
        lines_to_return.append(right_line)
    
    return np.array(lines_to_return)

In [7]:
# def draw_lanes(img, lines, color=[0, 255, 255], thickness=3):
#     try:
#         ys = []  
#         for i in lines:
#             for ii in i:
#                 ys += [ii[1], ii[3]]
#         min_y = min(ys)
#         max_y = 600
#         new_lines = []
#         line_dict = {}

#         for idx, i in enumerate(lines):
#             for xyxy in i:
#                 x_coords = (xyxy[0], xyxy[2])
#                 y_coords = (xyxy[1], xyxy[3])
#                 A = vstack([x_coords, ones(len(x_coords))]).T
#                 m, b = lstsq(A, y_coords)[0]

#                 x1 = (min_y - b) / m
#                 x2 = (max_y - b) / m

#                 line_dict[idx] = [m, b, [int(x1), min_y, int(x2), max_y]]
#                 new_lines.append([int(x1), min_y, int(x2), max_y])

#         final_lanes = {}

#         for idx in line_dict:
#             final_lanes_copy = final_lanes.copy()
#             m = line_dict[idx][0]
#             b = line_dict[idx][1]
#             line = line_dict[idx][2]
            
#             if len(final_lanes) == 0:
#                 final_lanes[m] = [[m, b, line]]
#             else:
#                 found_copy = False
#                 for other_ms in final_lanes_copy:
#                     if not found_copy:
#                         if abs(other_ms * 1.2) > abs(m) > abs(other_ms * 0.8):
#                             if abs(final_lanes_copy[other_ms][0][1] * 1.2) > abs(b) > abs(final_lanes_copy[other_ms][0][1] * 0.8):
#                                 final_lanes[other_ms].append([m, b, line])
#                                 found_copy = True
#                                 break
#                         else:
#                             final_lanes[m] = [[m, b, line]]

#         line_counter = {}

#         for lanes in final_lanes:
#             line_counter[lanes] = len(final_lanes[lanes])

#         top_lanes = sorted(line_counter.items(), key=lambda item: item[1])[::-1][:2]

#         lane1_id = top_lanes[0][0]
#         lane2_id = top_lanes[1][0]

#         def average_lane(lane_data):
#             x1s = []
#             y1s = []
#             x2s = []
#             y2s = []
#             for data in lane_data:
#                 x1s.append(data[2][0])
#                 y1s.append(data[2][1])
#                 x2s.append(data[2][2])
#                 y2s.append(data[2][3])
#             return int(mean(x1s)), int(mean(y1s)), int(mean(x2s)), int(mean(y2s)) 

#         l1_x1, l1_y1, l1_x2, l1_y2 = average_lane(final_lanes[lane1_id])
#         l2_x1, l2_y1, l2_x2, l2_y2 = average_lane(final_lanes[lane2_id])

#         return [l1_x1, l1_y1, l1_x2, l1_y2], [l2_x1, l2_y1, l2_x2, l2_y2], lane1_id, lane2_id
#     except Exception as e:
#         print(str(e))


In [8]:
# def process_img(image):
#     original_image = image
   
#     processed_img = canny(image)
    
#     cropped_image = region_of_interest(processed_img)
#     lines = cv2.HoughLinesP(cropped_image, 2, np.pi / 180, 100, minLineLength=40, maxLineGap=5)

#     # lines = cv2.HoughLinesP(processed_img, 1, np.pi / 180, 180, 20, 15)
#     m1 = 0
#     m2 = 0
#     try:
#         l1, l2, m1, m2 = draw_lanes(original_image, lines)
#         cv2.line(original_image, (l1[0], l1[1]), (l1[2], l1[3]), [0, 255, 0], 30)
#         cv2.line(original_image, (l2[0], l2[1]), (l2[2], l2[3]), [0, 255, 0], 30)
#     except Exception as e:
#         print(str(e))
#         pass
#     try:
#         for coords in lines:
#             coords = coords[0]
#             try:
#                 cv2.line(processed_img, (coords[0], coords[1]), (coords[2], coords[3]), [255, 0, 0], 3)
#             except Exception as e:
#                 print(str(e))
#     except Exception as e:
#         pass

#     return processed_img, original_image, m1, m2

In [9]:
# def straight():
#     # if random.randrange(0,3) == 1:
#     #     PressKey(W)
#     #     time.sleep(0.1)
#     # else:
#     #     ReleaseKey(W)
#     PressKey(W)
#     ReleaseKey(A)
#     ReleaseKey(D)
#     ReleaseKey(S)
#     # time.sleep(0.4)
#     # ReleaseKey(W)

# def left():
#     if random.randrange(0,3)==1:
#         PressKey(W) 
#     else:
#         ReleaseKey(W)
#     PressKey(A) 
#     time.sleep(0.4)
#     ReleaseKey(D)
#     ReleaseKey(S)
#     # ReleaseKey(A)

# def right():
#     if random.randrange(0,3)==1:
#         PressKey(W)
#     else:
#         ReleaseKey(W)   
#     PressKey(D) 
#     time.sleep(0.4)

#     ReleaseKey(A)
#     ReleaseKey(S)
#     # ReleaseKey(D)

In [10]:
# def main():
#     last_time = time.time()
#     while True:
#         screen = np.array(ImageGrab.grab(bbox=(0, 40, 800, 640)))
#         print('Frame took {} seconds'.format(time.time() - last_time))
#         last_time = time.time()
#         new_screen, original_image, m1, m2 = process_img(screen)
#         cv2.imshow('window2', cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))

#         if m1 < 0 and m2 < 0:
#             right()
#         elif m1 > 0 and m2 > 0:
#             left()
#         else:
#             straight()

#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             cv2.destroyAllWindows()
#             break


In [11]:
bbox=(0, 40, 800, 640)
def main():
    while True:
        screen = np.array(ImageGrab.grab(bbox=bbox)) #left, top, right, bottom
        frame = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
        
        canny_image = canny(frame)
        cropped_image = region_of_interest(canny_image)

        #(img, rho, theta, threshold, minLineLength,maxLineGap)
        lines = cv2.HoughLinesP(cropped_image, 2, np.pi / 180, 100, minLineLength=40, maxLineGap=5)
        if lines is not None:
            lines = [line[0] for line in lines]  # Flatten the lines array
        averaged_lines = average_slope_intercept(frame, lines)
        line_image = display_lines(frame, averaged_lines)
        combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
        cv2.imshow("result", combo_image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

In [13]:
main()